# SVM model for toxic YouTube comments

En este notebook entrenamos un modelo de **SVM (LinearSVC)** para detectar
comentarios de odio/toxicidad en YouTube.

Objetivos:

- Cargar el dataset **preprocesado** (texto limpio/lematizado + etiquetas).
- Definir columna de texto y columna objetivo (`IsToxic` u otra).
- Vectorizar el texto con **TF-IDF** (unigramas y bigramas).
- Entrenar un modelo **LinearSVC** con `class_weight="balanced"`.
- Evaluar el modelo (accuracy, precision, recall, f1, ROC-AUC).
- Guardar el modelo entrenado como **`.pkl`** en `models/`.
- Guardar las métricas en un **`.json`** en `results/` con el formato acordado.


## 1. Imports y configuración

Importamos todas las librerías necesarias para:

- Carga de datos y manejo de rutas (`pandas`, `pathlib`).
- Modelado (`scikit-learn`).
- Guardado de modelo (`joblib`) y métricas (`json`).
- Medición de tiempo y timestamp (`datetime`).


In [2]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import json
from datetime import datetime

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

import joblib

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 2. Carga del dataset preprocesado

Cargamos el CSV **ya preprocesado** generado en el notebook de preprocessing
(`preprocessing_eda.ipynb` o similar).

- Ajusta la ruta `PREPROCESSED_PATH` según el nombre real de tu fichero.
- Este CSV debería contener:
  - Una columna de texto procesado (`text_processed` o similar).
  - La columna objetivo (`IsToxic` u otra).
  - Opcionalmente, features numéricos diseñados a mano (longitud, mayúsculas, etc.).


In [3]:
# Ajusta esta ruta al nombre del CSV que generaste en preprocessing
PREPROCESSED_PATH = Path("../../data/preprocessing_data/youtoxic_english_1000_clean.csv")

df = pd.read_csv(PREPROCESSED_PATH)

print("Shape del dataset preprocesado:", df.shape)
df.head()


Shape del dataset preprocesado: (997, 18)


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsReligiousHate,text_basic,text_classic,text_len_classic,word_count_classic,uppercase_ratio,exclamation_count,hate_words_count
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,If only people would just take a step back and...,people would take step back make case wasnt an...,850,129,0.014121,0,2
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,Law enforcement is not trained to shoot to app...,law enforcement trained shoot apprehend traine...,90,13,0.036232,0,3
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\r\nDont you reckon them 'black lives matter' ...,True,True,False,False,True,False,False,False,Dont you reckon them 'black lives matter' bann...,dont reckon black life matter banner held whit...,252,40,0.002375,0,1
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,There are a very large number of people who do...,large number people like police officer called...,339,49,0.015464,0,0
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,"The Arab dude is absolutely right, he should h...",arab dude absolutely right shot extra time sho...,138,23,0.020576,0,1


## 3. Definición de columnas de texto, target y features numéricas

Nuestro dataset preprocesado incluye:

- `text_classic`: texto preprocesado pensado para **modelos clásicos**
  (TF-IDF, Naive Bayes, Regresión Logística, SVM, etc.).
- `text_basic`: texto más "ligero" para modelos **modernos** (embeddings, transformers, etc.).
- 5 features numéricas de apoyo:
  - `text_len_classic`
  - `word_count_classic`
  - `uppercase_ratio`
  - `exclamation_count`
  - `hate_words_count`

Como SVM es un modelo clásico, en este notebook usaremos **`text_classic`**
junto con esas 5 features numéricas.

También definimos la columna objetivo (`IsToxic`), que representa la tarea de
clasificación binaria (tóxico / no tóxico).


In [4]:
# Texto para modelos clásicos
TEXT_COL = "text_classic"

# Columna objetivo binaria (ajusta si usáis otra, por ejemplo IsAnyToxic)
TARGET_COL = "IsToxic"

# Features numéricas ya creadas en el preprocessing
NUMERIC_FEATURES = [
    "text_len_classic",
    "word_count_classic",
    "uppercase_ratio",
    "exclamation_count",
    "hate_words_count",
]

print("Comprobación de columnas:")
print("  TEXT_COL existe      :", TEXT_COL in df.columns)
print("  TARGET_COL existe    :", TARGET_COL in df.columns)
print("  Numeric features OK  :", [c for c in NUMERIC_FEATURES if c in df.columns])

# Eliminamos filas con target nulo por seguridad
df = df.dropna(subset=[TARGET_COL]).reset_index(drop=True)

X_text = df[TEXT_COL]
y = df[TARGET_COL].astype(int)

n_samples = df.shape[0]
print(f"\nNúmero de muestras utilizadas: {n_samples}")
print("\nDistribución de la clase objetivo:")
print(y.value_counts(normalize=True).sort_index())


Comprobación de columnas:
  TEXT_COL existe      : True
  TARGET_COL existe    : True
  Numeric features OK  : ['text_len_classic', 'word_count_classic', 'uppercase_ratio', 'exclamation_count', 'hate_words_count']

Número de muestras utilizadas: 997

Distribución de la clase objetivo:
IsToxic
0    0.539619
1    0.460381
Name: proportion, dtype: float64


## 4. Train/Test split estratificado

Dividimos el dataset en:

- `X_train_text`, `X_test_text`: textos de train y test.
- `y_train`, `y_test`: etiquetas de train y test.

Usamos estratificación por la columna objetivo para mantener la misma
proporción de tóxico/no tóxico en ambos conjuntos.


In [5]:
TEST_SIZE = 0.2

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y,
)

train_size_real = len(X_train_text) / n_samples
test_size_real = len(X_test_text) / n_samples

print(f"Tamaño train: {len(X_train_text)} ({train_size_real:.4f})")
print(f"Tamaño test: {len(X_test_text)} ({test_size_real:.4f})")


Tamaño train: 797 (0.7994)
Tamaño test: 200 (0.2006)


## 5. Pipeline de preprocesado y modelo: TF-IDF + numéricas + LinearSVC

Como nuestro dataset incluye tanto texto como features numéricas, construimos
un pipeline con dos partes:

1. **`ColumnTransformer` de preprocesado**:
   - Rama `text`: aplica `TfidfVectorizer` sobre la columna `text_classic`.
   - Rama `num`: aplica `StandardScaler` sobre las 5 features numéricas.
2. **Clasificador `LinearSVC`**:
   - Modelo SVM lineal con `class_weight="balanced"` para manejar mejor
     el posible desbalanceo de clases.

Este enfoque permite entrenar y usar el modelo con un único objeto (`Pipeline`)
que se serializa a `.pkl`.


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# Definimos el vectorizador TF-IDF para la columna de texto
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
)

# Preprocesador que combina texto (TF-IDF) y numéricas (escaladas)
preprocessor = ColumnTransformer(
    transformers=[
        ("text", tfidf_vectorizer, TEXT_COL),
        ("num", StandardScaler(with_mean=False), NUMERIC_FEATURES),
    ]
)

svm_clf = LinearSVC(
    class_weight="balanced",
    random_state=RANDOM_STATE,
)

svm_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("svm", svm_clf),
    ]
)

svm_pipeline


,steps,"[('preprocess', ...), ('svm', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('text', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


## 6. Entrenamiento del modelo SVM

Entrenamos el pipeline completo:

- `preprocess` ajusta TF-IDF sobre `text_classic` y escala las columnas numéricas.
- `svm` entrena el clasificador lineal sobre las features combinadas.


In [7]:
%%time

svm_pipeline.fit(df[[TEXT_COL] + NUMERIC_FEATURES].loc[X_train_text.index], y_train)


CPU times: total: 141 ms
Wall time: 141 ms


,steps,"[('preprocess', ...), ('svm', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('text', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


## 7. Evaluación del modelo en train y test

Calculamos accuracy, precision, recall, F1 y ROC-AUC para train y test,
usando el pipeline completo (texto + numéricas).


In [8]:
def evaluate_svm_model(pipeline, df, X_train_idx, y_train, X_test_idx, y_test):
    # Construimos los DataFrames de entrada (texto + numéricas)
    X_train_df = df[[TEXT_COL] + NUMERIC_FEATURES].loc[X_train_idx]
    X_test_df = df[[TEXT_COL] + NUMERIC_FEATURES].loc[X_test_idx]

    # Predicciones
    y_train_pred = pipeline.predict(X_train_df)
    y_test_pred = pipeline.predict(X_test_df)

    # Scores (para ROC-AUC)
    y_train_scores = pipeline.decision_function(X_train_df)
    y_test_scores = pipeline.decision_function(X_test_df)

    # Métricas train
    acc_train = accuracy_score(y_train, y_train_pred)
    prec_train, rec_train, f1_train, _ = precision_recall_fscore_support(
        y_train, y_train_pred, average="binary", zero_division=0
    )
    roc_train = roc_auc_score(y_train, y_train_scores)

    # Métricas test
    acc_test = accuracy_score(y_test, y_test_pred)
    prec_test, rec_test, f1_test, _ = precision_recall_fscore_support(
        y_test, y_test_pred, average="binary", zero_division=0
    )
    roc_test = roc_auc_score(y_test, y_test_scores)

    metrics_train = {
        "accuracy": acc_train,
        "precision": prec_train,
        "recall": rec_train,
        "f1": f1_train,
        "roc_auc": roc_train,
    }

    metrics_test = {
        "accuracy": acc_test,
        "precision": prec_test,
        "recall": rec_test,
        "f1": f1_test,
        "roc_auc": roc_test,
    }

    print("=== Métricas TRAIN ===")
    for k, v in metrics_train.items():
        print(f"{k:10s}: {v:.4f}")

    print("\n=== Métricas TEST ===")
    for k, v in metrics_test.items():
        print(f"{k:10s}: {v:.4f}")

    print("\nDiferencia accuracy train-test:", metrics_train["accuracy"] - metrics_test["accuracy"])
    print("Diferencia f1 train-test      :", metrics_train["f1"] - metrics_test["f1"])

    return metrics_train, metrics_test


metrics_train, metrics_test = evaluate_svm_model(
    svm_pipeline,
    df,
    X_train_text.index,
    y_train,
    X_test_text.index,
    y_test,
)


=== Métricas TRAIN ===
accuracy  : 0.9837
precision : 0.9810
recall    : 0.9837
f1        : 0.9823
roc_auc   : 0.9982

=== Métricas TEST ===
accuracy  : 0.7050
precision : 0.6941
recall    : 0.6413
f1        : 0.6667
roc_auc   : 0.7607

Diferencia accuracy train-test: 0.2786888331242159
Diferencia f1 train-test      : 0.31564625850340144


# 🔁 Celda 8 – Matriz de confusión (igual pero con DataFrame de entrada)

In [9]:
X_test_df = df[[TEXT_COL] + NUMERIC_FEATURES].loc[X_test_text.index]
y_test_pred = svm_pipeline.predict(X_test_df)

cm = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cm.ravel()

print("Matriz de confusión (tn, fp, fn, tp):")
print(cm)
print(f"\nTN={tn}, FP={fp}, FN={fn}, TP={tp}\n")

print("Classification report (TEST):")
print(classification_report(y_test, y_test_pred, digits=4))


Matriz de confusión (tn, fp, fn, tp):
[[82 26]
 [33 59]]

TN=82, FP=26, FN=33, TP=59

Classification report (TEST):
              precision    recall  f1-score   support

           0     0.7130    0.7593    0.7354       108
           1     0.6941    0.6413    0.6667        92

    accuracy                         0.7050       200
   macro avg     0.7036    0.7003    0.7010       200
weighted avg     0.7043    0.7050    0.7038       200



# 🔁 Celda 9 – Diccionario JSON (ajuste para n_features_text & numeric)

In [10]:
MODEL_NAME = "svm_toxic_v1"

# Accedemos al vectorizador TF-IDF dentro del ColumnTransformer
preprocess_step = svm_pipeline.named_steps["preprocess"]
tfidf = preprocess_step.named_transformers_["text"]
n_features_text = len(tfidf.get_feature_names_out())

n_features_numeric = len(NUMERIC_FEATURES)

results_dict = {
    "model_name": MODEL_NAME,
    "task": "binary_classification",
    "target_label": TARGET_COL,
    "data": {
        "n_samples": int(n_samples),
        "n_features_text": int(n_features_text),
        "n_features_numeric": int(n_features_numeric),
        "train_size": float(train_size_real),
        "test_size": float(test_size_real),
        "random_state": int(RANDOM_STATE),
    },
    "metrics": {
        "accuracy": float(metrics_test["accuracy"]),
        "precision": float(metrics_test["precision"]),
        "recall": float(metrics_test["recall"]),
        "f1": float(metrics_test["f1"]),
        "roc_auc": float(metrics_test["roc_auc"]),
    },
    "confusion_matrix": {
        "tn": int(tn),
        "fp": int(fp),
        "fn": int(fn),
        "tp": int(tp),
    },
    "timestamp": datetime.now().isoformat(),
    "notes": "LinearSVC + TF-IDF (1,2) + 5 numeric features on text_classic",
}

results_dict


{'model_name': 'svm_toxic_v1',
 'task': 'binary_classification',
 'target_label': 'IsToxic',
 'data': {'n_samples': 997,
  'n_features_text': 1136,
  'n_features_numeric': 5,
  'train_size': 0.7993981945837513,
  'test_size': 0.20060180541624875,
  'random_state': 42},
 'metrics': {'accuracy': 0.705,
  'precision': 0.6941176470588235,
  'recall': 0.6413043478260869,
  'f1': 0.6666666666666666,
  'roc_auc': 0.7606682769726247},
 'confusion_matrix': {'tn': 82, 'fp': 26, 'fn': 33, 'tp': 59},
 'timestamp': '2025-12-01T09:00:24.702598',
 'notes': 'LinearSVC + TF-IDF (1,2) + 5 numeric features on text_classic'}